# Extract

In [1]:
import pymssql
import pandas as pd
import json
import multiprocessing as mp
import datetime
import pymysql

In [2]:
server = '192.168.4.117'
database = 'FreedomCashLenders'
username = 'FreedomCashLendersAll'
mssql_password = 'Freedom123$'

In [3]:
iloans_conn = pymssql.connect(server, username, mssql_password, database, port = 1433)

In [4]:
query_loan = '''select LN.LoanId,
                       LN.OriginationDate,
                       GC.BankReportData,
                       LN.Campaign,
                       LN.MonthlyGrossIncome,
                       LN.DateOfBirth,
                       LN.IsFirstDefault
                       
                from view_FCL_Loan LN
                LEFT JOIN view_FCL_GetCreditDataLoan GCD ON LN.LoanId = GCD.LoanId
                LEFT JOIN view_FCL_GetCreditData GC ON GC.BankTransactionId = GCD.BankTransactionId
                
                
                where LN.OriginationDate >= '2018-01-01' 
                and LN.OriginationDate <= '2019-12-31' 
                and LN.IsFirstDefault IS NOT NULL
                and LN.MerchantId IN (15, 18)
                and GC.ReportStatus = 'COMPLETE' '''

In [5]:
df_loans = pd.read_sql_query(query_loan,con = iloans_conn)

In [6]:
df_loans = df_loans.drop_duplicates('LoanId')

# Pre-process

## utility functions

In [7]:
def parse_dates(json_date):
    '''
    Converts json formatted date to pandas datetime.
    
    Parameters:
    JSON date (JSON).
    
    Returns:
    Pandas datetime object.
    
    '''
    
    #return datetime.fromtimestamp(int(json_date)/1000.0).strftime('%Y-%m-%d')
    return datetime.datetime.utcfromtimestamp(int(json_date)/1000).date()


def fetch_checking_acct_txns(json_string):
    """
    Parse all checking account transactions in the bank report
    
    Parameters:
    json_string(json): json containing bank report
    
    Returns:
    dataframe: containing transactions 
    
    """
    j = json.loads(json_string)
    df_txn = pd.DataFrame()
    
    acct_numbers = []
    for accts in j['accounts']:
        
        if ('transactions' in accts.keys()) and (len(accts['transactions']) > 0) and (accts['accountNumber'] not in acct_numbers) and (accts['accountType'].strip().lower() == 'checking'):
            
            df_txn_temp = pd.DataFrame(accts['transactions'])
            df_txn_temp['account_number'] = accts['accountNumber']
            df_txn = df_txn.append(df_txn_temp, ignore_index=True)
            
            df_txn['posted_date'] = df_txn['postedDate'].map(lambda json_date: parse_dates(json_date))
            df_txn['category'] = df_txn['contexts'].map(lambda x: x[0]['categoryName'] if len(x) > 0 else np.nan)
            acct_numbers.append(accts['accountNumber'])
    
    if 'pending' in df_txn.columns:
        df_txn = df_txn[df_txn['pending'] == False]
    return df_txn

## primary account

In [8]:
def get_primary_account(bankreport):
    """
    Flag primary checking account (account having max transaction count)
    
    Parameters:
    bankreport (json)
    loanid (str)
    
    Returns:
    Dataframe containing checking accounts and primary account flag = 1
    """
    df_txn = fetch_checking_acct_txns(bankreport)
    if df_txn.empty is False:
        df_txns_count = df_txn['account_number'].value_counts()
        return df_txns_count.idxmax()

In [9]:
NCPU = mp.cpu_count() - 2 if mp.cpu_count() > 2 else 1
with mp.Pool(processes=NCPU) as pool:
    res_primary_accts = pool.map(get_primary_account, df_loans['BankReportData'])

/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of p

In [10]:
df_loans['primary_account'] = res_primary_accts

In [15]:
df_loans.columns

Index(['LoanId', 'OriginationDate', 'BankReportData', 'Campaign',
       'MonthlyGrossIncome', 'DateOfBirth', 'IsFirstDefault',
       'primary_account', 'Age'],
      dtype='object')

## calculate age

In [12]:
def calculate_age(current_date, dob):
    age = len(pd.date_range(start=dob,end=current_date,freq='Y'))
    return age

In [13]:
df_loans['Age'] = df_loans.apply(lambda x: calculate_age(x['OriginationDate'],x['DateOfBirth']), axis = 1)

## transaction days count

In [17]:
df_checking_txns=fetch_checking_acct_txns(df_loans['BankReportData'].iloc[0])

/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [18]:
df_checking_txns.head(3)

,account_number,amount,balance,category,checkNumber,contexts,institutionTxId,memo,merchant,pending,postedDate,posted_date,type
0,9774,-50.0,50.36,Transfer,None,"[{'categoryName': 'Transfer', 'categoryType': ...",8179999905,TRF TO DD 1101200377,,False,1545026400000,2018-12-17,debit
1,9774,100.0,100.36,Transfer,None,"[{'categoryName': 'Transfer', 'categoryType': ...",8179999906,AUTOMATIC TRANSFER,,False,1544767200000,2018-12-14,credit
2,9774,-20.0,0.36,Transfer,None,"[{'categoryName': 'Transfer', 'categoryType': ...",8179999907,TRF TO DD 1101200377,,False,1544680800000,2018-12-13,debit


In [19]:
df_primary_account_txns = df_checking_txns[df_checking_txns['account_number']=='9774']

In [20]:
 df_primary_account_txns= df_primary_account_txns.sort_values(by='posted_date')

In [27]:
first_txn_date = df_primary_account_txns['posted_date'].iloc[0]

In [28]:
last_txn_date = df_primary_account_txns['posted_date'].iloc[-1]

In [29]:
txn_days_count = (last_txn_date - first_txn_date).days

In [30]:
txn_days_count

87

In [49]:
def get_transaction_days_count(primary_account,bank_report):
    df_checking_txns = fetch_checking_acct_txns(bank_report)
    if df_checking_txns.empty is False:
        df_primary_account_txns = df_checking_txns[df_checking_txns['account_number']==primary_account]
        df_primary_account_txns= df_primary_account_txns.sort_values(by='posted_date')
        first_txn_date = df_primary_account_txns['posted_date'].iloc[0]
        last_txn_date = df_primary_account_txns['posted_date'].iloc[-1]
        txn_days_count = (last_txn_date - first_txn_date).days
        return txn_days_count >= 60

In [50]:
NCPU = mp.cpu_count() - 2 if mp.cpu_count() > 2 else 1
with mp.Pool(processes=NCPU) as pool:
    txn_days_count = pool.starmap(get_transaction_days_count, zip(df_loans['primary_account'],df_loans['BankReportData']))

/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of p

/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of p

/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of p

/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of p

/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of p

...

/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of p

/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of p

/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of p

/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of p

/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of p

In [51]:
df_loans['txn_days_count'] = txn_days_count

## new or reloan